In [1]:
    try:
        from google.colab import drive

        print('Colab environment detected. Mounting drive...')
        drive.mount('/content/drive')

        print('Mounted. Switching to directory... ', end = '')
        %cd /content/drive/'My Drive'/CILroadseg
        print('done.')
    except:
        print('Colab environment not found. Working on ordinary directory.')

Colab environment detected. Mounting drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted. Switching to directory... /content/drive/My Drive/CILroadseg
done.


In [2]:
import numpy as np
import sys
import os
import matplotlib.image as mpimg

from util.submit import *
from util.helpers import *
from util.visualize import *
from util.notebooks import *
from util.cross_validation import cross_validate

## Loading Training Data

In [3]:
train_dir = "training/images/"
gt_dir = "training/groundtruth/"
test_dir = "test/images/"

X, Y, X_test = nb_load_data(train_dir, gt_dir, test_dir)

Loading training input...
Progress: done (100 images).
Loading training groundtruth...
Progress: done (100 images).
Loading test input...
Progress: done (94 images).

       Training data shape: (100, 400, 400, 3)
Training groundtruth shape: (100, 400, 400)
           Test data shape: (94, 608, 608, 3)


## Decomposer + ResNet/CNN

In [4]:
import tensorflow as tf
from tensorflow import keras

from decomposer import *
from discretize import *
from rotate_vote import *
from zero_classifier import *
from cnn_new import *
from cnn import *

Using TensorFlow backend.


In [5]:
core_model = ConvNetModel()

model = Decomposer(Discretizer(core_model))
model_rnv = Decomposer(RotAndVote(Discretizer(core_model)))
model_rnm = Decomposer(RotAndVote(core_model, method='avg'))

In [ ]:
model_id = "convnet5"

choice = 'tr'

np.random.seed(1) #fix randomness
if choice == 'cv':
  cross_validate(model, 10, X, Y)

elif choice == 'tr':
  model.initialize()
  model.train(Y, X) 

  nb_save_model(model, "saves/final/"+ model_id +".h5") 

else:
  model.initialize()
  model.load("saves/final/"+ model_id +".h5")

u
Epoch 1/200
 - 69s - loss: 0.4712 - accuracy: 0.7816 - val_loss: 0.3382 - val_accuracy: 0.8681
Epoch 2/200


## Submission Generation

In [ ]:
from recomposer import *
model = Recomposer(model)
model_rnv = Recomposer(model_rnv)
model_rnm = Recomposer(model_rnm)

In [ ]:
test_masks_dir = "test/pred/"
test_dir = "test/images/"
 
nb_predict_masks(model, test_dir, test_masks_dir + "normal/")
nb_predict_masks(model_rnv, test_dir, test_masks_dir + "vote/")
nb_predict_masks(model_rnm, test_dir, test_masks_dir + "mean/")

In [ ]:
image_paths = [test_masks_dir + "normal/" + file for file in os.listdir(test_masks_dir + "normal/")]
masks_to_submission("test/"+ model_id +".csv", image_paths)

image_paths = [test_masks_dir + "vote/" + file for file in os.listdir(test_masks_dir + "vote/")]
masks_to_submission("test/"+ model_id +"+vote.csv", image_paths)

image_paths = [test_masks_dir + "mean/" + file for file in os.listdir(test_masks_dir + "mean/")]
masks_to_submission("test/"+ model_id +"+mean.csv", image_paths)

In [ ]:
Y_pred = model.classify(X_test[0:10])
Y_pred_rnv = model_rnv.classify(X_test[0:10])
Y_pred_rnm = model_rnm.classify(X_test[0:10])

view_image_array(X_test[0:10], Y_pred, Y_pred_rnv, Y_pred_rnm)